In [1]:
#from cassandra.cluster import Cluster
import pandas as pd 
import collections
import numpy as np

In [2]:
import csv

col_list = ["movieId", "title","genres"]

df = pd.read_csv("/home/spetz/Downloads/archive/movie.csv", usecols=col_list)
#file.readlines()
for_query2=df
df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [3]:
df=df.drop(columns=["genres"])


In [4]:
columns_list = ["userId", "movieId","rating","timestamp"]
dx=pd.read_csv("/home/spetz/Downloads/archive/rating.csv", usecols=columns_list)
dx=dx.sort_values("movieId")
dx=dx[:10000000]
dx

,userId,movieId,rating,timestamp
14162882,97809,1,3.0,2008-06-11 04:47:11
15347420,106140,1,5.0,2013-01-29 03:33:49
15347355,106138,1,3.0,2002-07-31 15:48:53
10170195,70354,1,4.5,2011-02-13 18:55:40
10170255,70355,1,3.5,2008-01-26 16:56:54
...,...,...,...,...
7816716,53914,2167,4.0,2003-04-09 00:25:00
8114556,55895,2167,4.0,2005-01-30 21:04:59
10750246,74315,2167,2.0,2000-04-05 02:33:11
19621413,135822,2167,1.0,2000-08-02 02:31:03


In [5]:
dx=dx.drop(columns=["userId"])
dx


,movieId,rating,timestamp
14162882,1,3.0,2008-06-11 04:47:11
15347420,1,5.0,2013-01-29 03:33:49
15347355,1,3.0,2002-07-31 15:48:53
10170195,1,4.5,2011-02-13 18:55:40
10170255,1,3.5,2008-01-26 16:56:54
...,...,...,...
7816716,2167,4.0,2003-04-09 00:25:00
8114556,2167,4.0,2005-01-30 21:04:59
10750246,2167,2.0,2000-04-05 02:33:11
19621413,2167,1.0,2000-08-02 02:31:03


In [6]:

dt = dx.sort_values('timestamp').drop_duplicates(['movieId','rating'], keep='last')
dt

,movieId,rating,timestamp
13768535,109,5.0,1996-03-19 04:44:10
19549902,625,5.0,1996-03-28 08:55:52
10594753,752,5.0,1996-05-17 19:08:16
10594712,655,5.0,1996-05-17 19:09:07
12054970,666,1.0,1996-05-22 10:44:15
...,...,...,...
8378393,924,4.5,2015-03-31 05:43:51
8378412,1653,3.5,2015-03-31 05:44:40
8378405,1240,5.0,2015-03-31 05:50:01
8378411,1590,3.5,2015-03-31 05:51:49


In [8]:
new_list = list()
dicts={}
dxx = dx[(dx['timestamp'] > '2015-01-01') & (dx['timestamp'] <= '2015-03-31')]
for i in range(len(dxx)):
    ar = dxx.loc[(dx['movieId'] == i ) & (dx['rating'] <= 5)]
    rating=ar.apply("rating").sum()
    movies_num=ar['movieId'].value_counts()
    x=sum(rating/movies_num)
    new_list.append(x)
    
    first_value = dxx['movieId'].iat[i]
    dicts[i] = new_list[i]




In [24]:
dicts={x:y for x,y in dicts.items() if y!=0}
#ERWTIMA3
def compare_dict_values(dictionary, given_float):
    return {i: dictionary[i] for i in dictionary if dictionary[i] > given_float}


final_table=compare_dict_values(dicts,0.1)
final_table = pd.DataFrame(list(final_table.items()), columns=['movieId', 'rating'])

final_table

,movieId,rating
0,1,3.860412
1,2,3.128834
2,3,2.854167
3,4,2.000000
4,5,2.824324
...,...,...
1554,2163,1.666667
1555,2164,2.500000
1556,2165,1.750000
1557,2166,3.625000


In [42]:
#QUERY1
df
first_query = df.merge(final_table, on="movieId", how = 'inner')
first_query=first_query.nlargest(30, ['rating'])
n=first_query.drop(columns=['movieId'])
#first_query

#ADD TIMESTAMP
dt = dxx.sort_values('timestamp').drop_duplicates(['movieId','rating'], keep='last')

f=dt.drop(['rating'], axis=1)
f
timequery = first_query.merge(f, on="movieId", how = 'inner')
timequery=timequery.nlargest(30,['rating'])
#final query1 with timestamp
timequery


,movieId,title,rating,timestamp
0,176,Living in Oblivion (1995),5.000000,2015-02-08 01:25:32
1,182,Moonlight and Valentino (1995),5.000000,2015-03-22 23:11:24
2,183,Mute Witness (1994),5.000000,2015-03-24 01:10:08
3,214,Before the Rain (Pred dozhdot) (1994),5.000000,2015-03-12 22:41:48
4,363,"Wonderful, Horrible Life of Leni Riefenstahl, ...",5.000000,2015-01-12 03:48:52
5,556,"War Room, The (1993)",5.000000,2015-01-31 22:01:00
6,823,"Collector, The (La collectionneuse) (1967)",5.000000,2015-01-10 21:13:12
7,964,Angel and the Badman (1947),5.000000,2015-02-28 06:27:24
8,1014,Pollyanna (1960),5.000000,2015-01-27 23:29:32
9,1547,Shiloh (1997),5.000000,2015-03-02 20:39:30


In [52]:
Query1 = pd.DataFrame(list(dicts.items()), columns=['movieId', 'rating'])

final_query = Query1.merge(df, on="movieId", how = 'inner')
final_query = final_query.dropna()

final_query = final_query.merge(f, on="movieId", how = 'inner')
final_query = final_query.sort_values('timestamp').drop_duplicates(['movieId','rating','title'], keep='last')


final_query

,movieId,rating,title,timestamp
5201,1313,3.500000,Mad Dog Time (1996),2015-01-01 17:20:23
2006,472,1.000000,I'll Do Anything (1994),2015-01-01 23:57:27
3583,963,3.500000,"Inspector General, The (1949)",2015-01-02 09:41:37
4234,1151,3.000000,Faust (1994),2015-01-02 12:16:46
990,228,3.000000,Destiny Turns on the Radio (1995),2015-01-02 22:02:01
...,...,...,...,...
2884,745,3.319328,Wallace & Gromit: A Close Shave (1995),2015-03-30 23:37:01
3356,913,3.807143,"Maltese Falcon, The (1941)",2015-03-30 23:37:24
3445,926,4.107143,All About Eve (1950),2015-03-30 23:38:09
5901,1527,3.896259,"Fifth Element, The (1997)",2015-03-30 23:58:48


In [55]:
#QUERYY2
for_query2


final_query2 = for_query2.merge(Query1, on="movieId", how = 'inner')

final_query2


,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.128834
2,3,Grumpier Old Men (1995),Comedy|Romance,2.854167
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.000000
4,5,Father of the Bride Part II (1995),Comedy,2.824324
...,...,...,...,...
1554,2163,Attack of the Killer Tomatoes! (1978),Comedy|Horror,1.666667
1555,2164,Surf Nazis Must Die (1987),Action|Comedy|Drama|Horror,2.500000
1556,2165,Your Friends and Neighbors (1998),Comedy|Drama,1.750000
1557,2166,Return to Paradise (1998),Crime|Drama|Romance|Thriller,3.625000


In [13]:

col_list=['userId','movieId','tag','timestamp']
dyy = pd.read_csv("/home/spetz/Downloads/archive/tag.csv", usecols=col_list)

In [14]:
dyy=dyy.drop(columns=['userId','timestamp'])
dyy
final_query22 = final_query2.merge(dyy, on="movieId", how = 'inner')

In [15]:
final_query22



,movieId,title,genres,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Watched
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,computer animation
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Disney animated feature
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Pixar animation
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,TÃ©a Leoni does not star in this movie
...,...,...,...,...,...
101812,2167,Blade (1998),Action|Horror|Thriller,3.069892,Netflix Finland
101813,2167,Blade (1998),Action|Horror|Thriller,3.069892,action
101814,2167,Blade (1998),Action|Horror|Thriller,3.069892,Bibliothek
101815,2167,Blade (1998),Action|Horror|Thriller,3.069892,vampire


In [16]:
#QUERY3
final_query3=final_query2.drop(columns=["movieId",'genres','rating'])


In [17]:
final_query3


,title
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
...,...
1554,Attack of the Killer Tomatoes! (1978)
1555,Surf Nazis Must Die (1987)
1556,Your Friends and Neighbors (1998)
1557,Return to Paradise (1998)


In [18]:
only_numbers=final_query3['title'].str.extract('(\d+)')
only_numbers.columns=['years']

In [19]:
only_numbers
final_query2['years']= only_numbers['years']
final_query2
query3=final_query2
query3


,movieId,title,genres,rating,years
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.128834,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,2.854167,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.000000,1995
4,5,Father of the Bride Part II (1995),Comedy,2.824324,1995
...,...,...,...,...,...
1554,2163,Attack of the Killer Tomatoes! (1978),Comedy|Horror,1.666667,1978
1555,2164,Surf Nazis Must Die (1987),Action|Comedy|Drama|Horror,2.500000,1987
1556,2165,Your Friends and Neighbors (1998),Comedy|Drama,1.750000,1998
1557,2166,Return to Paradise (1998),Crime|Drama|Romance|Thriller,3.625000,1998


In [20]:
#query4
query4=final_query22
query5=final_query22
query4

,movieId,title,genres,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Watched
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,computer animation
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Disney animated feature
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Pixar animation
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,TÃ©a Leoni does not star in this movie
...,...,...,...,...,...
101812,2167,Blade (1998),Action|Horror|Thriller,3.069892,Netflix Finland
101813,2167,Blade (1998),Action|Horror|Thriller,3.069892,action
101814,2167,Blade (1998),Action|Horror|Thriller,3.069892,Bibliothek
101815,2167,Blade (1998),Action|Horror|Thriller,3.069892,vampire


In [43]:

final_query.to_csv("/home/spetz/Downloads/archive/query1.csv")
final_query22.to_csv("/home/spetz/Downloads/archive/query2.csv")
query3.to_csv("/home/spetz/Downloads/archive/query3.csv")
query4.to_csv("/home/spetz/Downloads/archive/query4.csv")
query5.to_csv("/home/spetz/Downloads/archive/query5.csv")
timequery.to_csv("/home/spetz/Downloads/archive/query11.csv")

In [122]:
#CASSANDRA 

#CREATE KEYSPACE bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3};

CREATE TABLE query1(
        Unnamed int,
        title text,
        rating float,
        PRIMARY KEY(title,rating));

#COPY query1(Unnamed,title, rating) FROM '/home/spetz/Downloads/archive/query1.csv' WITH HEADER = true;

select * from query1;
###################################################################################################################################
#QUERY1 SELECT NEW

CREATE TABLE query11(
		Unnamed int,
        movieId int,
        title text,
        rating float,
		timestamp timestamp ,
        PRIMARY KEY(timestamp,rating));



#COPY query11(Unnamed,movieId,title, rating,timestamp) FROM '/home/spetz/Downloads/archive/query11.csv' WITH HEADER = true;

select title,rating from query11 where timestamp >= '2015-01-01 00:00:00.000000+0000' AND timestamp <'2015-01-15 00:00:00.000000+0000' ALLOW FILTERING;


       
###################################################################################################################################
CREATE TABLE query2(
        Unnamed int,
        movieId int,
        title text,
        genres text,
        rating float,
        tag text,
        PRIMARY KEY(movieId,title));

#COPY query2(Unnamed,movieId,title, genres,rating,tag) FROM '/home/spetz/Downloads/archive/query2.csv' WITH HEADER = true;

select * from query2 where title='Jumanji (1995)' ALLOW FILTERING;

###################################################################################################################################

CREATE TABLE query3(
        Unnamed int,
        movieId int,
        title text,
        genres text,
        rating float,
        years int,
        PRIMARY KEY(genres,years));
#COPY query3(Unnamed,movieId,title, genres,rating,years) FROM '/home/spetz/Downloads/archive/query3.csv' WITH HEADER = true; 
select title from query3 where genres='Adventure' order by years desc;

###################################################################################################################################
CREATE TABLE query4(
        Unnamed int,
        movieId int,
        title text,
        genres text,
        rating float,
        tag text,
        PRIMARY KEY((rating),title));

#COPY query4(Unnamed,movieId,title, genres,rating,tag) FROM '/home/spetz/Downloads/archive/query4.csv' WITH HEADER = true;
CREATE CUSTOM INDEX employee_firstname_idx ON query4 (title) USING 'org.apache.cassandra.index.sasi.SASIIndex' WITH OPTIONS = {'mode': 'CONTAINS', 'analyzer_class': 'org.apache.cassandra.index.sasi.analyzer.StandardAnalyzer', 'case_sensitive': 'false'};

select title from query4 WHERE title LIKE '%Blade%' ALLOW FILTERING;

###################################################################################################################################
CREATE TABLE query5(
        Unnamed int,
        movieId int,
        title text,
        genres text,
        rating float,
        tag text,
        PRIMARY KEY((tag),rating));

#COPY query5(Unnamed,movieId,title, genres,rating,tag) FROM '/home/spetz/Downloads/archive/query5.csv' WITH HEADER = true;

#Select 5
select title from query5 WHERE tag='Comedy' order by rating desc limit 20 ALLOW FILTERING;


SyntaxError: invalid syntax (1330531045.py, line 5)

In [25]:
from cassandra.cluster import Cluster
cluster =Cluster(['0.0.0.0'], port=9042)
session = cluster.connect("bigdata")